In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In order to start, we have to define the equation that defines the projectile motion of the ball. In order to do this I will define the forces that act on the ball seperately to simplify the final equation.

According to NASA (https://www.grc.nasa.gov/WWW/K-12/airplane/balldrag.html) the drag coefficient on a baseball is equal to 0.3
Similarly according to NASA (https://www.grc.nasa.gov/WWW/K-12/airplane/balllift.html) the lift coefficient on a baseball is 0.15

It is also important to note that a baseball's circumfrence and mass are not constant for all baseballs, instead they fall inside a tolerance. The range for radius is .229-.235 meters and the mass is .142-.149 kg (https://en.wikipedia.org/wiki/Baseball_(ball)#:~:targetText=Under%20the%20current%20rules%2C%20a,%E2%80%9376%20mm%20in%20diameter). We will just be using the middle of the range as an approximation; i.e .232 m and .1455 kg

For air density, we will just be using the values provided here (https://www.purplerow.com/2018/1/8/16846116/colorado-rockies-coors-field-arizona-diamondbacks-chase-field-comparison#:~:targetText=On%20a%20typical%20summer%20day,has%20an%20air%20density%20of%20.) This also provides us with an air density for San Diego which we will use as our control.



In [6]:
#Here we will be defining a bunch of constants used later in case we want to change them.
Cd= 0.3
Cl=0.15
mass= .1455
Rball = .232/(2*np.pi)
CrossA= np.pi* Rball**2
g=9.8
Airdensea= 1.182 
Airdenco= .966 

termvsea=(np.sqrt ( (2 * mass * g) / (Cd * Airdensea * CrossA) ))
termvco=(np.sqrt ( (2 * mass * g) / (Cd * Airdenco * CrossA) ))


In [12]:
def terminaldrag(Location): #Arg requires either co or sea for the two heights
    if Location == "co":
        return(.5*Cd*Airdenco*Aball*termvco**2)
    elif Location == "sea":
        return(.5*Cd*Airdensea*Aball*termvsea**2)

def drag(vel,Location):
    if Location == "co":
        return .5*Cd*Airdenco*Aball*vel**2
    elif Location == "sea":
        return .5*Cd*Airdensea*Aball*vel**2
    
def lift(vel,spin,Location):
    if Location == 'co':
        return(Cl*(4 * np.pi^2 * Rball**3 * spin * Airdenco * vel) * (4/3))
    elif Location == 'sea':
        return(Cl*(4 * np.pi^2 * Rball**3 * spin * Airdensea * vel) * (4/3))


def ascensionf(vel, Location):
    if Location == 'co':
        return( (-mass*g) - drag(vel, Airdenco))
    elif Location =="sea":
        return( (-mass*g) - drag(vel, Airdensea))